In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results, fem_analysis
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 2: Magnetic Shielding, Spherical Shell of Permeable Material in a Uniform Field

# 3-noded Linear Triangular Elements, $P_1$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 3)

BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'Az_0'}, 
            {'type': 'Dirichlet', 'tag': 102, 'value': 0.05, 'name': 'Az_0.05'}]

MATERIAL = [{'type': 'relative_magnetic_permeability', 'tag': 201, 'value': 1, 'name': 'cavity'},
            {'type': 'relative_magnetic_permeability', 'tag': 202, 'value': 1E4, 'name': 'magnetic_shell'},
            {'type': 'relative_magnetic_permeability', 'tag': 203, 'value': 1, 'name': 'free_space'}]

# Create mesh from file geometry
create_domain.axial_rectangular_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, lc=0.05, view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='axial_rectangular', info_mode=False)

Malha salva em pre_processing/mesh/axial_rectangular_domain_Triangle3.msh
Model axial_rectangular_domain_Triangle3 (2D)
Info     : 21 geometric entities
Info     : 5 Physical Groups
Info     : 13228 nodes in total
Info     : 2887 2-D elements in total


### Mesh Data Dictionaries

In [3]:
mesh_nodes = mesh_data['nodes_data']; mesh_nodes[:1]

[{'TagID': 1,
  'global_coord': (0.0, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}}]

In [4]:
conn_data = mesh_data['conn_data']; conn_data[:1]

[{'TagID': 110,
  'conn_list': [900, 946, 1011, 1029, 1030, 1031, 1032, 1033, 1034, 1035],
  'material': {'type': 'relative_magnetic_permeability',
   'tag': 201,
   'value': 1,
   'name': 'cavity'}}]

In [5]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

### Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='axial_rectangular', Numbering=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pre_processing\pictures\axial_rectangular_meshed_domain_Triangle3.svg


<figure>
    <img src="pre_processing/pictures/axial_rectangular_meshed_domain_Triangle3.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

#### `apply_physics()`

In [7]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 110,
  'conn_list': [900, 946, 1011, 1029, 1030, 1031, 1032, 1033, 1034, 1035],
  'material': {'type': 'relative_magnetic_permeability',
   'tag': 201,
   'value': 1,
   'name': 'cavity',
   'a_value': 795774.7150262763},
  'source': {'type': 'free_source', 'value': 0}}]

#### Material Propriety, $a$

In [8]:
a = [element['material']['a_value'] for element in mesh_data['conn_data']]
print("Material 'a' values: \n", np.array(a))

Material 'a' values: 
 [795774.71502628 795774.71502628 795774.71502628 ... 795774.71502628
 795774.71502628 795774.71502628]


### Imposition of Boundary Conditions

In [9]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")

The entire domain has 13228 nodes: 12899 free nodes and 329 Dirichlet nodes.


## Processor Module

### Reduced Global matrix $A_{gr}$ with boundary conditions

In [10]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (12899, 12899)


### Global Potential Vector

In [11]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)
print("Reduced solution vector shape: ", ur.shape)
print("Global potential vector shape: ", potential_u.shape)

Reduced solution vector shape:  (12899,)
Global potential vector shape:  (13228,)


# Magnetic Potencial Vector, $A_{\varphi}$

## Post-Processor

In [12]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='axial_rectangular')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pos_processing\pictures\axial_rectangular_domain_solution_Triangle3.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/axial_rectangular_domain_solution_Triangle3.svg" alt="Fig.2" style="width:60%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### `magnetic_field()`

In [13]:
B_field = fem_analysis.magnetic_field_density(mesh_data, FINITE_ELEMENT, potential_u)
graph_results.magnetic_field_distribution(mesh_data, B_field, FINITE_ELEMENT)
graph_results.magnetic_flux_lines(mesh_data, B_field, FINITE_ELEMENT)
graph_results.lateral_profile(mesh_data, B_field, FINITE_ELEMENT, y=0.0)
graph_results.diagonal_profile(mesh_data, B_field, FINITE_ELEMENT)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pos_processing\pictures\b_field_Triangle3.svg
Linhas de fluxo magnético salvas em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pos_processing\pictures\flux_lines_Triangle3.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pos_processing\pictures\lateral_b_profile_Triangle3.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p22_magnetic_shell_axial\pos_processing\pictures\diagonal_b_profile_Triangle3.svg


### Figure 3: Magnetic field density distribution

<figure>
    <img src="pos_processing/pictures/b_field_Triangle3.svg" alt="Fig.3" style="width:60%;" />
    <figcaption>Figure 3: Magnetic field density distribution.</figcaption>
</figure>

### Figure 4: Magnetic flux lines

<figure>
    <img src="pos_processing/pictures/flux_lines_Triangle3.svg" alt="Fig.4" style="width:60%;" />
    <figcaption>Figure 4: Magnetic flux lines.</figcaption>
</figure>

### Figure 5: Magnetic density field radial profile

<figure>
    <img src="pos_processing/pictures/lateral_b_profile_Triangle3.svg" alt="Fig.5" style="width:60%;" />
    <figcaption>Figure 5: Magnetic field density radial profile.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.